<a href="https://colab.research.google.com/github/fabuloudy/diploma/blob/main/dataset_generation(%D1%80%D1%83%D1%81%D1%81%D0%BA%D0%B8%D0%B9).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# На русском

In [ ]:
!pip install transformers 

In [ ]:
!pip install torch

## Загрузка модели ru-GPT3

In [ ]:

from transformers import GPT2LMHeadModel, GPT2Tokenizer
model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()
text = "Александр Сергеевич Пушкин родился в "
input_ids = tokenizer.encode(text, return_tensors="pt").cuda()
out = model.generate(input_ids.cuda())
generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Александр Сергеевич Пушкин родился в 
1799 году в Москве.

В 1820 году,


## Запуск модели для теста, первичный взгляд :)

In [ ]:
text = "Путин больше не президент "
input_ids = tokenizer.encode(text, return_tensors="pt").cuda()
out = model.generate(input_ids.cuda(),max_length = 150, min_length= 50,temperature = 1,top_p=0.95,do_sample=True,top_k = 10,num_beam = 2,repetition_penalty=5.0)
generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Путин больше не президент??
А кто это?<s>:))) А как же тогда, аааа... "ВСЁ" в России...)) ) И всё - от Путина. В смысле все-все!)))) Не так уж и много на самом деле)). Все зависит только о того с какой скоростью Путин будет двигаться к этому статусу (как бы он его ни старался сохранить), то есть если за год ничего такого сделать или хотябы постараться достичь у нас еще можно было добиться.... Но ведь уже поздно что либо менять.) Так вот теперь мы видим всю глубину падения уровня жизни населения нашей страны во всем ее многообразии по сравнению даже со многими странами Европы!!! Мы просто поражены этой бедой до глубины души!!!! Как говорится : НЕ


## Подбор наиболее хороших параметров среди: top_k, top_p, temp

Получилось около тысячи комбинаций, смотрела на одном и том же заголовке. Методом пристального взгляда выбрала наиболее подходящие и близкие к статейный (статьи про генерацию, там иногда проскальзывают подходящие чиселки)

In [ ]:
with open('article.csv','a') as fd:
  texts = [
    "В ингушском городе Малгобек произошло два мощных взрыва."
  ]
  top_k = [10,20,30,40,50,60,70,80,90,100]
  top_p=[0.85,0.87,0.89,0.90,0.92,0.94,0.96,0.98,1.0,1.05,2.0,3.0]
  temp = [0.7,0.8,0.9,1.0,1.1,1.2,2.0,3.0,4.0,5.0,10.0]
  for k in top_k:
    for p in top_p:
      for tem in temp:
        for text in texts:

          input_ids = tokenizer.encode(text, return_tensors="pt").cuda()
          out = model.generate(input_ids.cuda(), min_length = 80, max_length = 100, do_sample=True, top_k=k, top_p=p, temperature=tem)
          generated_text = list(map(tokenizer.decode, out))[0]
          balise = generated_text.find('<s>')
          text = str()
          if (balise != -1):
            text = generated_text[:balise]
          else:
            text = generated_text
          print(text)
          re = "k={} p={} temp={}".format(k,p,tem)
          fd.write(text)
          fd.write(re)
          fd.write("\n")
  
 человекk=10 p=0.85 temp=1.1
 вk=10 p=0.85 temp=2.0
 
  
    

## Загрузка файла со статейными заголовками и оригинальными текстами

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving true_data.csv to true_data.csv


In [ ]:
print(df['title'])

0       Географов отправят в Иорданию учиться добывать...
1       Европейский суд уравнял гомосексуальное партне...
2       1914. Еврейский «Голос Труда» выступил в защит...
3       В ингушском городе Малгобек произошло два мощн...
4       Philips научился делать жидкие линзы с характе...
                              ...                        
1995    Ходорковский будет зарабатывать в колонии две ...
1996    Зрители утреннего шоу разглядели в небе инопла...
1997    СМИ сообщили о причастности «Аль-Каиды» к тера...
1998          В Молдавии прекратили трансляцию «России 1»
1999    "Манчестер Юнайтед" выиграл у "Челси" в чемпио...
Name: title, Length: 2000, dtype: object


In [ ]:
import csv 
import pandas as pd
df = pd.read_csv("true_data.csv")
category = list(df['topic'])
true_text = list(df['text'])
texts = list(df['title'])

columns=['Title','Topic', 'Data_generate','Original_data']
data = list()


## Генерация и сохранение датасета

In [ ]:
count = 0
k=10 
p=0.85
temp=1.1
for text in texts:
    input_ids = tokenizer.encode(text, return_tensors="pt").cuda()
    out = model.generate(input_ids.cuda(), min_length = 80, max_length = 100, do_sample=True, top_k=k, top_p=p, temperature=tem)
    generated_text = list(map(tokenizer.decode, out))[0]
    balise = generated_text.find('<s>')
    generated = str()
    if (balise != -1):
        generated = generated_text[:balise]
    else:
      generated = generated_text
    data.append((texts[count],category[count],generated,true_text[count]))
    count = count + 1
    print(count)

In [ ]:
table = pd.DataFrame(data, columns=columns)

In [ ]:
table.to_csv('data.csv', index=False)

In [ ]:
print(table)

                                               Title  ...                                      Original_data
0  Географов отправят в Иорданию учиться добывать...  ...  Экспедиция Water Diaries («Дневники воды») отп...

[1 rows x 4 columns]


# На английском

## Загрузка заголовков

In [ ]:
f = open("/gdrive/My Drive/True.csv",mode='r')

import csv
import re

true_data = list()

def preprocess(string):
     
     if string.find('Reuters'):
        return string[string.find('-')+2:]
     return string
 
def csv_reader(file_obj):
    reader = csv.reader(file_obj)
    for line in reader:
        true_data.append((line[0],preprocess(line[1])))
      
    
csv_reader(f)

In [ ]:
print(true_data[5])

## Загрузка модели для генерации

In [ ]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

#get large GPT2 tokenizer and GPT2 model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")
GPT2 = TFGPT2LMHeadModel.from_pretrained("gpt2-large", pad_token_id=tokenizer.eos_token_id)

In [ ]:
#for reproducability
SEED = 34

#maximum number of words in output text
MAX_LEN = 70

GPT2.summary()

In [ ]:
input_sequence = true_data[5][0]

def split_text(string):
    return ' '.join(string.split()[0:2*MAX_LEN])

In [ ]:
import tensorflow as tf
tf.random.set_seed(SEED)

import pandas as pd   #  pip install pandas

columns=['Number', 'Data_generate','Original_data']
data = list()


## Генерация части датасета 

Запускала несколько раз, т.к. не знала, что на GPU он мне датасет размеромм 2К сгенерит за 15 минут. Я 200 текстов ждала по 2 часа :)))
На русском, конечно, я уже была в курсе

In [ ]:
for i in range(400,600):
  
  input_sequence = true_data[i][0]
  input_ids = tokenizer.encode(input_sequence, return_tensors='tf')
  sample_outputs = GPT2.generate(
                              input_ids,
                              do_sample = True, 
                              max_length = 2*MAX_LEN,                              #to test how long we can generate and it be coherent
                              temperature = .7,
                              top_k = 50, 
                              top_p = 0.85, 
                              num_return_sequences = 1
  )
  data.append((i,tokenizer.decode(sample_outputs[0],skip_special_tokens = True),split_text(true_data[i][1])))
  print("successful"+str(i))
  


print(len(data))

## Cохранение данных

In [ ]:
df = pd.DataFrame(data, columns=columns)
df.to_csv(r'/gdrive/My Drive/output0612_data.csv')

In [ ]:
# set return_num_sequences > 1
beam_outputs = GPT2.generate(
    input_ids, 
    max_length = MAX_LEN, 
    num_beams = 5, 
    no_repeat_ngram_size = 2, 
    num_return_sequences = 5, 
    early_stopping = True
)

print('')
print("Output:\n" + 100 * '-')

# now we have 3 output sequences
for i, beam_output in enumerate(beam_outputs):
      print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

In [ ]:
from transformers import RobertaConfig, RobertaModel, RobertaTokenizer, RobertaForSequenceClassification



config = RobertaConfig.from_pretrained('roberta-base')
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification(config)